In [5]:
# make sure this is running in the right environment!! caiman, NOT caiman_test!
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from scipy.ndimage import gaussian_filter
from tifffile.tifffile import imwrite,imread
from skimage import measure as sm

import SimpleITK as sitk
import h5py

import largestinteriorrectangle as lir
import matplotlib.patches as pac
from copy import deepcopy

from scipy import io as sio
from scipy import interpolate as si
from scipy import signal as sg
from scipy import stats as ss
from scipy import spatial as sp
from skimage import measure as sm
from skimage import metrics as sme
from skimage import transform as st
from skimage import feature as sf
from scipy.sparse import csc_matrix

from tqdm.auto import tqdm,trange
#%load_ext memory_profiler

from pathlib import Path
import pickle

import cv2
import multiprocessing
from joblib import Parallel, delayed
from joblib.externals.loky import get_reusable_executor

import caiman as cm
from caiman.utils.visualization import nb_view_patches3d
from caiman.source_extraction.cnmf import cnmf,params
from caiman.utils.visualization import view_quilt,get_contours
from caiman.paths import caiman_datadir

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

import bokeh.plotting as bpl
bpl.output_notebook()

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.WARNING)

import time
from sklearn.cluster import HDBSCAN
from scipy.spatial.distance import pdist,squareform
from itertools import chain

from IPython.display import clear_output
from scipy.ndimage import center_of_mass
import scipy

import pickle

def load_pickle(file_path):
    """
    Load a dictionary from a pickle file.

    Args:
    - file_path (str): Path to the pickle file.

    Returns:
    - dict: Loaded dictionary.
    """
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

Loading BokehJS ...

In [6]:
#optimizing things from here - actually just stick with joblib/loky and take the L on timing - might not even be a CPU issue but data saving?
import contextlib
import joblib
from tqdm import tqdm

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [7]:
# load in all movies to be used in this set of figures:
# paired/matched dippings - ref movie and all after, exclude pre-ref for the few non-001 ref probes
# paired/matched invivo movies (LV only, exclude parenchymal for now)
# paired/matched invivo dipping
# paired/matched slice movies
# paired/matched slide dipping

movie_sets_dipping = [[r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells101\preprocessing\20230915_PL52_wells101_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells201\preprocessing\20230915_PL52_wells201_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells301\preprocessing\20230915_PL52_wells301_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230915_PL52\wells401\preprocessing\20230915_PL52_wells401_Yael_Done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL48\wells001\preprocessing\20230916_PL48_wells001_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL48\wells101\preprocessing\20230916_PL48_wells101_Tiara_done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells101\preprocessing\20230916_PL54_wells101_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells201\preprocessing\20230916_PL54_wells201_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells301\preprocessing\20230916_PL54_wells301_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells401\preprocessing\20230916_PL54_wells401_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230916_PL54\wells501\preprocessing\20230916_PL54_wells501_jordan_done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230920_PL58\wells001\preprocessing\20230920_PL58_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230920_PL58\wells001\preprocessing\20230920_PL58_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230920_PL58\wells001\preprocessing\20230920_PL58_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230920_PL58\wells101\preprocessing\20230920_PL58_wells101_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230920_PL58\wells201\preprocessing\20230920_PL58_wells201_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230920_PL58\wells301\preprocessing\20230920_PL58_wells301_Done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells101\preprocessing\20230922_PL28_wells101_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells201\preprocessing\20230922_PL28_wells201_maddy_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells301\preprocessing\20230922_PL28_wells301_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells401\preprocessing\20230922_PL28_wells401_Tiara_done_qc_done',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells501\preprocessing\20230922_PL28_wells501_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells601\preprocessing\20230922_PL28_wells601_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230922_PL28\wells701\preprocessing\20230922_PL28_wells701_david_done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells101\preprocessing\20230923_PL35_wells101_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells201\preprocessing\20230923_PL35_wells201_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells301\preprocessing\20230923_PL35_wells301_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells401\preprocessing\20230923_PL35_wells401_Yael_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230923_PL35\wells501\preprocessing\20230923_PL35_wells501_Yael_Done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells101\preprocessing\20230925_PL58_wells101_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells201\preprocessing\20230925_PL58_wells201_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells301\preprocessing\20230925_PL58_wells301_Tiara_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230925_PL58\wells401\preprocessing\20230925_PL58_wells401_Tiara_done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells101\preprocessing\20230926_PL56_wells101_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells201\preprocessing\20230926_PL56_wells201_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells301\preprocessing\20230926_PL56_wells301_jordan_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells401\preprocessing\20230926_PL56_wells401_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230926_PL56\wells501\preprocessing\20230926_PL56_wells501_david_done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL34\wells001\preprocessing\20230927_PL34_wells001_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL34\wells001\preprocessing\20230927_PL34_wells001_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL34\wells001\preprocessing\20230927_PL34_wells001_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL34\wells101\preprocessing\20230927_PL34_wells101_Done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL34\wells201\preprocessing\20230927_PL34_wells201_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL34\wells301\preprocessing\20230927_PL34_wells301_Done_qc_doneqc'],
                     [r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL58\wells001\preprocessing\20230927_PL58_wells001_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL58\wells001\preprocessing\20230927_PL58_wells001_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL58\wells001\preprocessing\20230927_PL58_wells001_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL58\wells101\preprocessing\20230927_PL58_wells101_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL58\wells201\preprocessing\20230927_PL58_wells201_david_done_qc_doneqc',
                     r'\\NASQUATCH\data\2p\peter\dipping\20230927_PL58\wells301\preprocessing\20230927_PL58_wells301_david_done_qc_doneqc']]
pythonic_dipping = [np.arange(len(movie_sets_dipping[i])/2).astype('int').tolist()*2 for i in range(len(movie_sets_dipping))]

movie_list_invivo = [r'\\NASQUATCH\data\2p\peter\in_vivo\220216_PK25PL8\mouse\preprocessing\220216_PK25PL8_mouse_david_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220311_PK24PL9\mouse\preprocessing\220311_PK24PL9_mouse_Tiara_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220406_PK24PL10\mouse\preprocessing\220406_PK24PL10_mouse_jordan_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220408_PK27PL20\mouse\preprocessing\220408_PK27PL20_mouse_maddy_done_qc_done',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220427_PK27PL20\mouse\preprocessing\220427_PK27PL20_mouse_Tiara_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220622_PK44PL17\mouse\preprocessing\220622_PK44PL17_mouse_Tiara_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220624_PK45PL13\mouse\preprocessing\220624_PK45PL13_mouse_Tiara_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220701_PK49PL18\mouse\preprocessing\220701_PK49PL18_mouse_jordan_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\220903_PK63PL34\mouse\preprocessing\220903_PK63PL34_mouse_Done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\221018_PS10PL24\mouse\preprocessing\221018_PS10PL24_mouse_Yael_Done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\230519_DG41PL52\mouse\preprocessing\230519_DG41PL52_mouse_Done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\230525_DG42PL39\mouse\preprocessing\230525_DG42PL39_mouse_Tiara_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\230526_PK84PL50\mouse\preprocessing\230526_PK84PL50_mouse_Yael_Done_qc_done',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\230726_DG47PL54\mouse\preprocessing\230726_DG47PL54_mouse_jordan_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\230802_DG48PL58\mouse\preprocessing\230802_DG48PL58_mouse_Done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\231018_DG52PL66\mouse\preprocessing\231018_DG52PL66_mouse_Tiara_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\231024_DG54PL61\mouse\preprocessing\231024_DG54PL61_mouse_Done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\231025_DG56PL62\mouse\preprocessing\231025_DG56PL62_mouse_jordan_done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\231030_DG60PL67\mouse\preprocessing\231030_DG60PL67_mouse_Yael_Done_qc_doneqc',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\231105_DG39PL32\mouse\preprocessing\231105_DG39PL32_mouse_maddy_done_qc_done',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\231107_DG44PL58\mouse\preprocessing\231107_DG44PL58_mouse_maddy_done_qc_done',
                    r'\\NASQUATCH\data\2p\peter\in_vivo\231108_DG61PL62\mouse\preprocessing\231108_DG61PL62_mouse_maddy_done_qc_done']

movie_list_invivo_d = [r'\\NASQUATCH\data\2p\peter\in_vivo\220216_PK25PL8\wells101\preprocessing\220216_PK25PL8_wells101_david_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220311_PK24PL9\wells101\preprocessing\220311_PK24PL9_wells101_Tiara_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220406_PK24PL10\wells101\preprocessing\220406_PK24PL10_wells101_jordan_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220408_PK27PL20\wells101\preprocessing\220408_PK27PL20_wells101_maddy_done_qc_done',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220427_PK27PL20\wells101\preprocessing\220427_PK27PL20_wells101_Tiara_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220622_PK44PL17\wells101\preprocessing\220622_PK44PL17_wells101_Tiara_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220624_PK45PL13\wells101\preprocessing\220624_PK45PL13_wells101_Tiara_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220701_PK49PL18\wells101\preprocessing\220701_PK49PL18_wells101_jordan_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\220903_PK63PL34\wells101\preprocessing\220903_PK63PL34_wells101_Done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\221018_PS10PL24\wells101\preprocessing\221018_PS10PL24_wells101_Yael_Done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\230519_DG41PL52\wells101\preprocessing\230519_DG41PL52_wells101_Done_qc_done',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\230525_DG42PL39\wells101\preprocessing\230525_DG42PL39_wells101_Tiara_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\230526_PK84PL50\wells101\preprocessing\230526_PK84PL50_wells101_Yael_Done_qc_done',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\230726_DG47PL54\wells101\preprocessing\230726_DG47PL54_wells101_jordan_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\230802_DG48PL58\wells101\preprocessing\230802_DG48PL58_wells101_Done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\231018_DG52PL66\wells101\preprocessing\231018_DG52PL66_wells101_Tiara_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\231024_DG54PL61\wells\preprocessing\231024_DG54PL61_wells_maddy_done_qc_done',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\231025_DG56PL62\wells\preprocessing\231025_DG56PL62_wells_jordan_done_qc_done',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\231030_DG60PL67\wells\preprocessing\231030_DG60PL67_wells_maddy_done_qc_done',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\231105_DG39PL32\wells\preprocessing\231105_DG39PL32_wells_maddy_done_qc_done',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\231107_DG44PL58\wells\preprocessing\231107_DG44PL58_wells_david_done_qc_doneqc',
                      r'\\NASQUATCH\data\2p\peter\in_vivo\231108_DG61PL62\wells\preprocessing\231108_DG61PL62_wells_maddy_done_qc_done']

movie_list_slice = [r'\\NASQUATCH\data\2p\peter\slice\20230509_DA1PL53\slice\preprocessing\20230509_DA1PL53_slice_Yael_Done_qc_done_m',
                   r'\\NASQUATCH\data\2p\peter\slice\20230607_SL1PL47\slice\preprocessing\20230607_SL1PL47_slice_Done_qc_doneqc_m',
                   r'\\NASQUATCH\data\2p\peter\slice\20230801_SL1PL35\slice\preprocessing\20230801_SL1PL35_slice_maddy_done_qc_done_m',
                   r'\\NASQUATCH\data\2p\peter\slice\20230817_SL1PL55\slice\preprocessing\20230817_SL1PL55_slice_maddy_done_qc_done_m', #
                   r'\\NASQUATCH\data\2p\peter\slice\20230817_SL1PL55\slice\preprocessing\20230817_SL1PL55_slice_maddy_done_qc_done_m', #
                   r'\\NASQUATCH\data\2p\peter\slice\20230822_SL2PL52\slice\preprocessing\20230822_SL2PL52_slice_maddy_done_qc_done_m',
                   r'\\NASQUATCH\data\2p\peter\slice\20230824_SL1PL60\slice\preprocessing\20230824_SL1PL60_slice_charlotte_done_qc_doneqc_m', #
                   r'\\NASQUATCH\data\2p\peter\slice\20230824_SL1PL60\slice\preprocessing\20230824_SL1PL60_slice_charlotte_done_qc_doneqc_m', #
                   r'\\NASQUATCH\data\2p\peter\slice\20230829_SL1PL56\slice\preprocessing\20230829_SL1PL56_slice_Tiara_done_qc_done_m',
                   r'\\NASQUATCH\data\2p\peter\slice\20230830_SL1PL28\slice\preprocessing\20230830_SL1PL28_slice_maddy_done_qc_done_m',
                   r'\\NASQUATCH\data\2p\peter\slice\20230831_SL1PL55\slice\preprocessing\20230831_SL1PL55_slice_Yael_Done_qc_done_m']

movie_list_slice_d = [r'\\NASQUATCH\data\2p\peter\slice\20230509_DA1PL53\wells101\preprocessing\20230509_DA1PL53_wells101_Yael_Done_qc_doneqc_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230607_SL1PL47\wells001\preprocessing\20230607_SL1PL47_wells001_Done_qc_doneqc_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230801_SL1PL35\wells101\preprocessing\20230801_SL1PL35_wells101_maddy_done_qc_doneqc_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230817_SL1PL55\wells001\preprocessing\20230817_SL1PL55_wells001_maddy_done_qc_done_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230817_SL1PL55\wells101\preprocessing\20230817_SL1PL55_wells101_maddy_done_qc_done_m', #pythonic!
                     r'\\NASQUATCH\data\2p\peter\slice\20230822_SL2PL52\wells001\preprocessing\20230822_SL2PL52_wells001_maddy_done_qc_doneqc_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230824_SL1PL60\wells001\preprocessing\20230824_SL1PL60_wells001_charlotte_done_qc_doneqc_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230824_SL1PL60\wells101\preprocessing\20230824_SL1PL60_wells101_charlotte_done_qc_doneqc_m', #pythonic!
                     r'\\NASQUATCH\data\2p\peter\slice\20230829_SL1PL56\wells001\preprocessing\20230829_SL1PL56_wells001_done_qc_done_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230830_SL1PL28\wells001\preprocessing\20230830_SL1PL28_wells001_Done_qc_doneqc_m',
                     r'\\NASQUATCH\data\2p\peter\slice\20230831_SL1PL55\wells001\preprocessing\20230831_SL1PL55_wells001_Yael_Done_qc_doneqc_m']
pythonic_slice = [0,0,0,0,1,0,0,1,0,0,0]

In [8]:
cnmf_vals_dipping = []

for k in trange(len(movie_sets_dipping)):
    cnmf_vals_dipping_n = []
    movie_sets_dipping_n = movie_sets_dipping[k]
    pythonic_dipping_n = pythonic_dipping[k]
    
    for l in trange(len(movie_sets_dipping_n)):

        cnmf_path = os.path.join(movie_sets_dipping_n[l], f'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
        if pythonic_dipping_n[l]==0:
            cnmf_m_path_matched = os.path.join(movie_sets_dipping_n[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched.hdf5')
        else:
            cnmf_m_path_matched = os.path.join(movie_sets_dipping_n[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched{pythonic_dipping_n[l]:01}.hdf5')
            
        vcnmf = cnmf.load_CNMF(cnmf_path,
                              n_processes=1,
                              dview=None)
        
        #vcnmf_m = cnmf.load_CNMF(cnmf_m_path,
        #                        n_processes=1,
        #                        dview=None)
        
        vcnmf_m_matched = cnmf.load_CNMF(cnmf_m_path_matched,
                                        n_processes=1,
                                        dview=None)
    
        cnmf_vals_dipping_n.append([vcnmf.estimates.C.shape,vcnmf_m_matched.estimates.C.shape]) #,vcnmf_m.estimates.C.shape
    
    cnmf_vals_dipping.append(cnmf_vals_dipping_n)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
cnmf_vals_dipping

[[[(183, 36), (61, 36)],
  [(183, 36), (55, 36)],
  [(183, 36), (54, 36)],
  [(183, 36), (56, 36)],
  [(177, 30), (61, 30)],
  [(178, 63), (55, 63)],
  [(212, 51), (54, 51)],
  [(155, 39), (56, 39)]],
 [[(184, 51), (108, 51)], [(191, 81), (108, 81)]],
 [[(166, 36), (63, 36)],
  [(166, 36), (64, 36)],
  [(166, 36), (55, 36)],
  [(166, 36), (53, 36)],
  [(166, 36), (56, 36)],
  [(193, 30), (63, 30)],
  [(185, 51), (64, 51)],
  [(175, 39), (55, 39)],
  [(179, 30), (53, 30)],
  [(179, 63), (56, 63)]],
 [[(133, 51), (71, 51)],
  [(133, 51), (73, 51)],
  [(133, 51), (60, 51)],
  [(127, 30), (71, 30)],
  [(121, 39), (73, 39)],
  [(117, 63), (60, 63)]],
 [[(181, 51), (82, 51)],
  [(181, 51), (99, 51)],
  [(181, 51), (77, 51)],
  [(181, 51), (96, 51)],
  [(181, 51), (77, 51)],
  [(181, 51), (67, 51)],
  [(181, 51), (78, 51)],
  [(196, 30), (82, 30)],
  [(175, 39), (99, 39)],
  [(173, 63), (77, 63)],
  [(184, 36), (96, 36)],
  [(204, 30), (77, 30)],
  [(203, 30), (67, 30)],
  [(198, 63), (78, 63

In [10]:
cnmf_vals_invivo = []

for l in trange(len(movie_list_invivo)):
    cnmf_path = os.path.join(movie_list_invivo[l], f'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
    #cnmf_m_path = os.path.join(movie_list_invivo[l], f'ch0_means_movie_nobg_cnmf_merged_m_refit.hdf5')
    cnmf_m_path_matched = os.path.join(movie_list_invivo[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched.hdf5')
    
    vcnmf = cnmf.load_CNMF(cnmf_path,
                          n_processes=1,
                          dview=None)
    
    #vcnmf_m = cnmf.load_CNMF(cnmf_m_path,
    #                        n_processes=1,
    #                        dview=None)
    
    vcnmf_m_matched = cnmf.load_CNMF(cnmf_m_path_matched,
                                    n_processes=1,
                                    dview=None)

    cnmf_vals_invivo.append([vcnmf.estimates.C.shape,vcnmf_m_matched.estimates.C.shape]) #,vcnmf_m.estimates.C.shape

  0%|          | 0/22 [00:00<?, ?it/s]

In [11]:
cnmf_vals_invivo_d = []

for l in trange(len(movie_list_invivo_d)):
    cnmf_path = os.path.join(movie_list_invivo_d[l], f'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
    #cnmf_m_path = os.path.join(movie_list_invivo[l], f'ch0_means_movie_nobg_cnmf_merged_m_refit.hdf5')
    cnmf_m_path_matched = os.path.join(movie_list_invivo_d[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched.hdf5')
    
    vcnmf = cnmf.load_CNMF(cnmf_path,
                          n_processes=1,
                          dview=None)
    
    #vcnmf_m = cnmf.load_CNMF(cnmf_m_path,
    #                        n_processes=1,
    #                        dview=None)
    
    vcnmf_m_matched = cnmf.load_CNMF(cnmf_m_path_matched,
                                    n_processes=1,
                                    dview=None)

    cnmf_vals_invivo_d.append([vcnmf.estimates.C.shape,vcnmf_m_matched.estimates.C.shape]) #,vcnmf_m.estimates.C.shape

  0%|          | 0/22 [00:00<?, ?it/s]

In [12]:
cnmf_vals_invivo

[[(121, 125), (34, 125)],
 [(215, 114), (52, 114)],
 [(165, 132), (39, 132)],
 [(120, 137), (70, 137)],
 [(218, 144), (74, 144)],
 [(172, 146), (57, 146)],
 [(236, 151), (71, 151)],
 [(185, 91), (74, 91)],
 [(111, 127), (48, 127)],
 [(229, 62), (70, 62)],
 [(123, 137), (55, 137)],
 [(178, 142), (84, 142)],
 [(148, 124), (65, 124)],
 [(208, 142), (80, 142)],
 [(195, 142), (69, 142)],
 [(172, 142), (99, 142)],
 [(140, 130), (68, 130)],
 [(174, 149), (59, 149)],
 [(202, 70), (101, 70)],
 [(184, 169), (85, 169)],
 [(164, 169), (103, 169)],
 [(158, 169), (80, 169)]]

In [13]:
cnmf_vals_invivo_d

[[(81, 42), (34, 42)],
 [(193, 120), (52, 120)],
 [(161, 114), (39, 114)],
 [(148, 81), (70, 81)],
 [(227, 153), (74, 153)],
 [(201, 165), (57, 165)],
 [(199, 15), (71, 15)],
 [(251, 159), (74, 159)],
 [(125, 156), (48, 156)],
 [(223, 174), (70, 174)],
 [(93, 51), (55, 51)],
 [(208, 18), (84, 18)],
 [(144, 24), (65, 24)],
 [(190, 45), (80, 45)],
 [(171, 114), (69, 114)],
 [(218, 69), (99, 69)],
 [(179, 69), (68, 69)],
 [(167, 69), (59, 69)],
 [(206, 69), (101, 69)],
 [(206, 69), (85, 69)],
 [(214, 69), (103, 69)],
 [(169, 69), (80, 69)]]

In [14]:
cnmf_vals_slice = []

for l in trange(len(movie_list_slice)):
    cnmf_path = os.path.join(movie_list_slice[l], f'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
    cnmf_m_path = os.path.join(movie_list_slice[l], f'ch0_means_movie_nobg_cnmf_merged_m_refit.hdf5')

    if pythonic_slice[l]==0:
        cnmf_m_path_matched = os.path.join(movie_list_slice[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched.hdf5')
    else:
        cnmf_m_path_matched = os.path.join(movie_list_slice[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched{pythonic_slice[l]:01}.hdf5')
    
    vcnmf = cnmf.load_CNMF(cnmf_path,
                          n_processes=1,
                          dview=None)
    
    vcnmf_m = cnmf.load_CNMF(cnmf_m_path,
                            n_processes=1,
                            dview=None)
    
    vcnmf_m_matched = cnmf.load_CNMF(cnmf_m_path_matched,
                                    n_processes=1,
                                    dview=None)

    cnmf_vals_slice.append([vcnmf.estimates.C.shape,vcnmf_m.estimates.C.shape,vcnmf_m_matched.estimates.C.shape])

  0%|          | 0/11 [00:00<?, ?it/s]

In [15]:
cnmf_vals_slice_d = []

for l in trange(len(movie_list_slice_d)):
    cnmf_path = os.path.join(movie_list_slice_d[l], f'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
    cnmf_m_path = os.path.join(movie_list_slice_d[l], f'ch0_means_movie_nobg_cnmf_merged_m_refit.hdf5')

    if pythonic_slice[l]==0:
        cnmf_m_path_matched = os.path.join(movie_list_slice_d[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched.hdf5')
    else:
        cnmf_m_path_matched = os.path.join(movie_list_slice_d[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched{pythonic_slice[l]:01}.hdf5')
    
    vcnmf = cnmf.load_CNMF(cnmf_path,
                          n_processes=1,
                          dview=None)
    
    vcnmf_m = cnmf.load_CNMF(cnmf_m_path,
                            n_processes=1,
                            dview=None)
    
    vcnmf_m_matched = cnmf.load_CNMF(cnmf_m_path_matched,
                                    n_processes=1,
                                    dview=None)

    cnmf_vals_slice_d.append([vcnmf.estimates.C.shape,vcnmf_m.estimates.C.shape,vcnmf_m_matched.estimates.C.shape])

  0%|          | 0/11 [00:00<?, ?it/s]

In [16]:
cnmf_vals_slice

[[(115, 26), (62, 26), (33, 26)],
 [(153, 29), (105, 29), (44, 29)],
 [(174, 53), (142, 53), (27, 53)],
 [(160, 54), (129, 54), (67, 54)],
 [(160, 54), (129, 54), (62, 54)],
 [(203, 56), (166, 56), (73, 56)],
 [(235, 53), (195, 53), (60, 53)],
 [(235, 53), (195, 53), (56, 53)],
 [(144, 56), (123, 56), (62, 56)],
 [(186, 42), (172, 42), (41, 42)],
 [(239, 56), (185, 56), (101, 56)]]

In [17]:
cnmf_vals_slice_d

[[(182, 24), (98, 24), (33, 24)],
 [(171, 60), (110, 60), (44, 60)],
 [(176, 153), (119, 153), (27, 153)],
 [(137, 36), (116, 36), (67, 36)],
 [(147, 78), (118, 78), (62, 78)],
 [(266, 81), (209, 81), (73, 81)],
 [(150, 36), (131, 36), (60, 36)],
 [(258, 78), (162, 78), (56, 78)],
 [(130, 105), (109, 105), (62, 105)],
 [(183, 105), (156, 105), (41, 105)],
 [(214, 105), (149, 105), (101, 105)]]

In [18]:
%%time
# pickle everything up and move to IW
basedir = r'\\nasquatch\data\2p\peter\grin_videos\s12_outputs'

with open(basedir+r'\cnmf_vals_dipping.pickle', 'wb') as f:
    pickle.dump(cnmf_vals_dipping, f)

with open(basedir+r'\cnmf_vals_invivo.pickle', 'wb') as f:
    pickle.dump(cnmf_vals_invivo, f)

with open(basedir+r'\cnmf_vals_invivo_d.pickle', 'wb') as f:
    pickle.dump(cnmf_vals_invivo_d, f)

with open(basedir+r'\cnmf_vals_slice.pickle', 'wb') as f:
    pickle.dump(cnmf_vals_slice, f)

with open(basedir+r'\cnmf_vals_slice_d.pickle', 'wb') as f:
    pickle.dump(cnmf_vals_slice_d, f)

CPU times: total: 0 ns
Wall time: 22 ms


In [45]:
cnmf_vals_slice = []

for l in trange(len(movie_list_slice)):
    cnmf_path = os.path.join(movie_list_slice[l], f'ch0_means_movie_nobg_cnmf_merged_refit.hdf5')
    cnmf_m_path = os.path.join(movie_list_slice[l], f'ch0_means_movie_nobg_cnmf_merged_m_refit.hdf5')
    cnmf_m_path_matched = os.path.join(movie_list_slice[l], f'ch0_means_movie_nobg_cnmf_merged_refit_matched.hdf5')
    
    vcnmf = cnmf.load_CNMF(cnmf_path,
                          n_processes=1,
                          dview=None)
    
    vcnmf_m = cnmf.load_CNMF(cnmf_m_path,
                            n_processes=1,
                            dview=None)
    
    vcnmf_m_matched = cnmf.load_CNMF(cnmf_m_path_matched,
                                    n_processes=1,
                                    dview=None)

    cnmf_vals_slice.append([vcnmf.estimates.C.shape,vcnmf_m.estimates.C.shape,vcnmf_m_matched.estimates.C.shape])

  0%|          | 0/1 [00:00<?, ?it/s]